In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/busbra/BUSBRA/5-fold-cv.csv
/kaggle/input/busbra/BUSBRA/bus_data.csv
/kaggle/input/busbra/BUSBRA/LICENSE.txt
/kaggle/input/busbra/BUSBRA/10-fold-cv.csv
/kaggle/input/busbra/BUSBRA/Images/bus_0897-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0913-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0592-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0638-l.png
/kaggle/input/busbra/BUSBRA/Images/bus_0386-l.png
/kaggle/input/busbra/BUSBRA/Images/bus_0402-l.png
/kaggle/input/busbra/BUSBRA/Images/bus_0173-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0668-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0785-s.png
/kaggle/input/busbra/BUSBRA/Images/bus_0918-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0258-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0884-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0141-l.png
/kaggle/input/busbra/BUSBRA/Images/bus_0121-s.png
/kaggle/input/busbra/BUSBRA/Images/bus_0362-r.png
/kaggle/input/busbra/BUSBRA/Images/bus_0565-r.png
/kaggle/input/busbra/BUSBRA/Images

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast
import albumentations as A

# ================================
# 1. Paths
# ================================
BASE_DIR = "/kaggle/input/busbra/BUSBRA"
IMG_DIR = f"{BASE_DIR}/Images"
MASK_DIR = f"{BASE_DIR}/Masks"
CSV_PATH = f"{BASE_DIR}/bus_data.csv"

OUT_DIR = "/kaggle/working/busbra_preprocessed"
os.makedirs(OUT_DIR, exist_ok=True)
for sub in ["fic", "bbc", "mic"]:
    os.makedirs(f"{OUT_DIR}/{sub}", exist_ok=True)

# ================================
# 2. Helpers
# ================================
def create_bbc(img, bbox, margin=30):
    x, y, w, h = bbox
    x_min = max(0, x - margin)
    y_min = max(0, y - margin)
    x_max = min(img.shape[1], x + w + margin)
    y_max = min(img.shape[0], y + h + margin)
    return img[y_min:y_max, x_min:x_max]

def create_mic(img, mask):
    masked = img.copy()
    masked[mask > 0] = 0
    return masked

augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5)
])

# ================================
# 3. Process dataset
# ================================
df = pd.read_csv(CSV_PATH)

records = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    img_name = row.iloc[0] + ".png" 
    mask_name = "mask_" + img_name.split("_")[1]  # mask_0001-r.png etc.

    img_path = f"{IMG_DIR}/{img_name}"
    mask_path = f"{MASK_DIR}/{mask_name}"

    if not os.path.exists(img_path) or not os.path.exists(mask_path):
        continue

    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    birads = int(row["BIRADS"])
    bbox = ast.literal_eval(row["BBOX"])  # [x,y,w,h]

    fic = cv2.resize(img, (224, 224))
    bbc = cv2.resize(create_bbc(img, bbox), (224, 224))
    mic = cv2.resize(create_mic(img, mask), (224, 224))

    fic_out = f"{OUT_DIR}/fic/{row.iloc[0]}.png"
    bbc_out = f"{OUT_DIR}/bbc/{row.iloc[0]}.png"
    mic_out = f"{OUT_DIR}/mic/{row.iloc[0]}.png"

    cv2.imwrite(fic_out, fic)
    cv2.imwrite(bbc_out, bbc)
    cv2.imwrite(mic_out, mic)

    records.append([fic_out, bbc_out, mic_out, birads])

df_out = pd.DataFrame(records, columns=["fic_path", "bbc_path", "mic_path", "birads"])
df_out.to_csv(f"{OUT_DIR}/busbra_preprocessed.csv", index=False)
print("✅ Preprocessing done, CSV saved.")

# ================================
# 4. Balance classes with augmentation
# ================================
class_counts = df_out["birads"].value_counts().to_dict()
max_count = max(class_counts.values())

aug_records = []
for birads in class_counts.keys():
    df_class = df_out[df_out["birads"] == birads]
    needed = max_count - len(df_class)

    if needed > 0:
        for i in range(needed):
            row = df_class.sample(1).iloc[0]
            fic = cv2.imread(row["fic_path"])
            bbc = cv2.imread(row["bbc_path"])
            mic = cv2.imread(row["mic_path"])

            fic_aug = augment(image=fic)["image"]
            bbc_aug = augment(image=bbc)["image"]
            mic_aug = augment(image=mic)["image"]

            aug_id = f"{birads}_aug{i}"
            fic_path = f"{OUT_DIR}/fic/{aug_id}.png"
            bbc_path = f"{OUT_DIR}/bbc/{aug_id}.png"
            mic_path = f"{OUT_DIR}/mic/{aug_id}.png"

            cv2.imwrite(fic_path, fic_aug)
            cv2.imwrite(bbc_path, bbc_aug)
            cv2.imwrite(mic_path, mic_aug)

            aug_records.append([fic_path, bbc_path, mic_path, birads])

if aug_records:
    df_aug = pd.DataFrame(aug_records, columns=["fic_path", "bbc_path", "mic_path", "birads"])
    df_out = pd.concat([df_out, df_aug], ignore_index=True)
    df_out.to_csv(f"{OUT_DIR}/busbra_balanced.csv", index=False)

print("✅ Dataset balanced & saved.")


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
100%|██████████| 1875/1875 [00:39<00:00, 47.62it/s]


✅ Preprocessing done, CSV saved.
✅ Dataset balanced & saved.


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import albumentations as A
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import torchvision.models as models
import random

# ================================
# 1. Paths
# ================================
BASE_DIR = "/kaggle/input/busbra/BUSBRA"
CSV_PATH = "/kaggle/working/busbra_preprocessed/busbra_balanced.csv"
OUT_DIR = "/kaggle/working/busbra_models"
os.makedirs(OUT_DIR, exist_ok=True)

# ================================
# 2. Augmentation (for balancing)
# ================================
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.RandomBrightnessContrast(p=0.3),
    A.Affine(translate_percent=0.05, scale=(0.9,1.1), rotate=(-15,15), p=0.5)
])

# ================================
# 3. Load CSV & Balance classes
# ================================
df = pd.read_csv(CSV_PATH)
class_counts = df['birads'].value_counts()
max_count = class_counts.max()

records = []
for birads in class_counts.index:
    df_class = df[df['birads'] == birads]
    records.extend(df_class.to_dict('records'))  # original samples

    # Oversample with augmentation
    needed = max_count - len(df_class)
    if needed > 0:
        for _ in range(needed):
            sample = random.choice(df_class.to_dict('records'))
            new_record = sample.copy()
            # Mark these as augmented for filenames
            new_record['fic_path'] = new_record['fic_path'].replace('.png', f'_aug.png')
            new_record['bbc_path'] = new_record['bbc_path'].replace('.png', f'_aug.png')
            new_record['mic_path'] = new_record['mic_path'].replace('.png', f'_aug.png')
            records.append(new_record)

df_balanced = pd.DataFrame(records)
df_balanced.to_csv("/kaggle/working/busbra_balanced_augmented.csv", index=False)
print(f"✅ Dataset balanced: {df_balanced.shape[0]} samples")

# ================================
# 4. Dataset Class
# ================================
class BUSBRADataset(Dataset):
    def __init__(self, df, transform=None, augment=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        fic = cv2.imread(row["fic_path"])
        bbc = cv2.imread(row["bbc_path"])
        mic = cv2.imread(row["mic_path"])

        fic = cv2.cvtColor(fic, cv2.COLOR_BGR2RGB)
        bbc = cv2.cvtColor(bbc, cv2.COLOR_BGR2RGB)
        mic = cv2.cvtColor(mic, cv2.COLOR_BGR2RGB)

        if self.augment:
            fic = augment(image=fic)['image']
            bbc = augment(image=bbc)['image']
            mic = augment(image=mic)['image']

        if self.transform:
            fic = self.transform(image=fic)['image']
            bbc = self.transform(image=bbc)['image']
            mic = self.transform(image=mic)['image']

        fic = torch.from_numpy(fic).permute(2,0,1).float() / 255.0
        bbc = torch.from_numpy(bbc).permute(2,0,1).float() / 255.0
        mic = torch.from_numpy(mic).permute(2,0,1).float() / 255.0

        label = torch.tensor(row["birads"] - 2, dtype=torch.long)
        return fic, bbc, mic, label, row["fic_path"]  # return path for tracking

# ================================
# 5. Load CSV
# ================================
num_classes = df_balanced["birads"].nunique()

# ================================
# 6. Model
# ================================
class MultiInputResNet50(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()
        self.fc = nn.Sequential(
            nn.Linear(2048*3, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, fic, bbc, mic):
        f1 = self.backbone(fic)
        f2 = self.backbone(bbc)
        f3 = self.backbone(mic)
        x = torch.cat([f1,f2,f3], dim=1)
        x = self.fc(x)
        return x

# ================================
# 7. Train / Eval
# ================================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    all_labels, all_preds = [], []
    total_loss = 0
    for fic, bbc, mic, labels, _ in tqdm(loader):
        fic, bbc, mic, labels = fic.to(device), bbc.to(device), mic.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(fic, bbc, mic)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    return total_loss/len(loader.dataset), acc, prec, rec, f1, classification_report(all_labels, all_preds, zero_division=0)

def evaluate(model, loader, criterion, device):
    model.eval()
    all_labels, all_preds, all_probs, all_files = [], [], [], []
    total_loss = 0
    softmax = nn.Softmax(dim=1)

    with torch.no_grad():
        for fic, bbc, mic, labels, paths in loader:
            fic, bbc, mic, labels = fic.to(device), bbc.to(device), mic.to(device), labels.to(device)
            outputs = model(fic, bbc, mic)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)

            probs = softmax(outputs)
            preds = probs.argmax(dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.max(dim=1).values.cpu().numpy())  # confidence
            all_files.extend(paths)

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    report = classification_report(all_labels, all_preds, zero_division=0)

    preds_df = pd.DataFrame({
        "image": all_files,
        "true_label": all_labels,
        "pred_label": all_preds,
        "confidence": all_probs
    })

    return total_loss/len(loader.dataset), acc, prec, rec, f1, report, preds_df

# ================================
# 8. 5-Fold Training
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
max_epochs = 200
batch_size = 16
patience = 15
fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(skf.split(df_balanced, df_balanced["birads"]), 1):
    print(f"\n===== Fold {fold} =====")
    train_df, val_df = df_balanced.iloc[train_idx], df_balanced.iloc[val_idx]

    train_dataset = BUSBRADataset(train_df, transform=None, augment=True)
    val_dataset = BUSBRADataset(val_df, transform=None, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    model = MultiInputResNet50(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    best_f1 = 0
    trigger_times = 0

    for epoch in range(1, max_epochs+1):
        train_loss, train_acc, train_prec, train_rec, train_f1, train_report = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc, val_prec, val_rec, val_f1, val_report, preds_df = evaluate(model, val_loader, criterion, device)

        print(f"Epoch {epoch}/{max_epochs} | "
              f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} F1: {train_f1:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f}")
        print("\nValidation Classification Report:\n", val_report)

        # Save per-fold validation predictions
        preds_df.to_csv(f"{OUT_DIR}/val_predictions_fold{fold}.csv", index=False)

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), f"{OUT_DIR}/best_model_fold{fold}.pth")
            trigger_times = 0
        else:
            trigger_times += 1

        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

    fold_metrics.append((fold, val_acc, val_prec, val_rec, val_f1))

# ================================
# 9. Summary
# ================================
print("\n===== 5-Fold Summary =====")
for m in fold_metrics:
    print(f"Fold {m[0]} | Acc: {m[1]:.4f} | Prec: {m[2]:.4f} | Rec: {m[3]:.4f} | F1: {m[4]:.4f}")


✅ Dataset balanced: 2772 samples

===== Fold 1 =====


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 200MB/s]
100%|██████████| 139/139 [00:41<00:00,  3.34it/s]


Epoch 1/200 | Train Loss: 1.0203 Acc: 0.5372 F1: 0.5355 | Val Loss: 0.7906 Acc: 0.6270 F1: 0.6281

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.68      0.70       139
           1       0.58      0.50      0.54       139
           2       0.50      0.83      0.62       139
           3       0.93      0.50      0.65       138

    accuracy                           0.63       555
   macro avg       0.68      0.63      0.63       555
weighted avg       0.68      0.63      0.63       555



100%|██████████| 139/139 [00:40<00:00,  3.45it/s]


Epoch 2/200 | Train Loss: 0.7536 Acc: 0.6743 F1: 0.6741 | Val Loss: 0.8507 Acc: 0.6198 F1: 0.6167

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.80      0.73       139
           1       0.57      0.53      0.54       139
           2       0.51      0.72      0.59       139
           3       0.98      0.43      0.60       138

    accuracy                           0.62       555
   macro avg       0.68      0.62      0.62       555
weighted avg       0.68      0.62      0.62       555



...

Epoch 98/200 | Train Loss: 0.0351 Acc: 0.9878 F1: 0.9878 | Val Loss: 0.9726 Acc: 0.8231 F1: 0.8259

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.76      0.79       139
           1       0.71      0.81      0.75       139
           2       0.81      0.84      0.82       138
           3       1.00      0.88      0.94       138

    accuracy                           0.82       554
   macro avg       0.83      0.82      0.83       554
weighted avg       0.83      0.82      0.83       554



100%|██████████| 139/139 [00:40<00:00,  3.46it/s]


Epoch 99/200 | Train Loss: 0.0348 Acc: 0.9865 F1: 0.9865 | Val Loss: 0.9587 Acc: 0.8177 F1: 0.8202

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.81      0.79       139
           1       0.70      0.79      0.74       139
           2       0.84      0.76      0.80       138
           3       0.98      0.91      0.95       138

    accuracy                           0.82       554
   macro avg       0.83      0.82      0.82       554
weighted avg       0.83      0.82      0.82       554



100%|██████████| 139/139 [00:40<00:00,  3.44it/s]


Epoch 100/200 | Train Loss: 0.0460 Acc: 0.9851 F1: 0.9851 | Val Loss: 0.7701 Acc: 0.8141 F1: 0.8163

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.72      0.77       139
           1       0.70      0.81      0.75       139
           2       0.79      0.83      0.81       138
           3       0.98      0.89      0.94       138

    accuracy                           0.81       554
   macro avg       0.82      0.81      0.82       554
weighted avg       0.82      0.81      0.82       554

Early stopping at epoch 100

===== 5-Fold Summary =====
Fold 1 | Acc: 0.8018 | Prec: 0.8147 | Rec: 0.8020 | F1: 0.8047
Fold 2 | Acc: 0.7730 | Prec: 0.8133 | Rec: 0.7727 | F1: 0.7753
Fold 3 | Acc: 0.8014 | Prec: 0.8037 | Rec: 0.8012 | F1: 0.8019
Fold 4 | Acc: 0.7599 | Prec: 0.7946 | Rec: 0.7601 | F1: 0.7636
Fold 5 | Acc: 0.8141 | Prec: 0.8236 | Rec: 0.8143 | F1: 0.8163
